# Momentum regression

This assignment aims to learn how to define and run a regression model for reconstructing the momentum of neutrino events. Regression models are typical in machine learning, where the algorithm learns how to estimate a **continuous value** from a vector of features.

##Prerequisites

Let's start with downloading the dataset, as well as loading the needed Python packages and modules:

In [ ]:
!wget "https://raw.githubusercontent.com/saulam/neutrinoml/main/modules.py"
!wget "https://raw.githubusercontent.com/saulam/neutrinoml/main/df_pgun_teaching.p"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import scale, PolynomialFeatures
from sklearn.model_selection import train_test_split
from modules import plot_event, plot_regression, plot_parameters, plot_distributions

##Dataset

We can now load the dataset:

In [ ]:
# read dataframe
df = pd.read_pickle('df_pgun_teaching.p')

We may have a look at the dataset. It consists of 59,578 particle gun events with the following attributes:

- **TruePID**: PDG code for particle identification (PID); 2212 (proton), 13 (muon), 211 (pion).
- **TrueMomentum**: momentum in MeV.
- **NNodes**: number of nodes of the event (3D spatial points).
- **NodeOrder**: order of the nodes within the event.
- **NodePosX**: array with the coordinates of the nodes along the X-axis (in mm).
- **NodePosY**: array with the coordinates of the nodes along the Y-axis (in mm).
- **NodePosZ**: array with the coordinates of the nodes along the Z-axis (in mm).
- **NodeT**: array with the timestamps of the nodes (in ms).
- **Nodededx**: array with energy deposits of the nodes (dE/dx).
- **TrkLen**: length of the track (in mm).
- **TrkEDepo**: total track energy deposition (in arbitrary unit).
- **TrkDir1**: track direction, polar angle (in degrees).
- **TrkDir2**: track direction, azimuth angle (in degrees).


In [ ]:
df

The 3D spatial points of the events are usually stored in the form of hits or nodes. We chose the latter for our dataset. A hit corresponds with a cube with real energy deposition (there are usually many hits across the track signature), whilst a node corresponds with a fitted position after performing the track reconstruction.

<div>
<img src="https://raw.githubusercontent.com/saulam/neutrinoml/main/hit.png" height="400"/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src="https://raw.githubusercontent.com/saulam/neutrinoml/main/node.png" height="400"/>
</div>

We may also have a look at the events by plotting the nodes within the detector space. By default, we're looking at the first event (event 0), but we can display more events by playing with the variable `event_number`.

In [ ]:
event_number = 0
plot_event(df, event_number)

Regardless of the type of data we use and the algorithm chosen, it is essential to perform a **preprocessing** of the data, which allows us to prepare the data to make it understandable for the machine-learning algorithm.

As explained before, the goal is to learn to estimate continuous value **y** from a fixed-size vector of features **X**. However, the input data is in 3D, and every event (track) has a different size. Thus, the simplest way of preprocessing our data is to use only the dE/dx of the first node of each track as input:

In [ ]:
X = np.zeros(shape=(len(df),1), dtype=np.float32) # array of size (n_event, 1)
y = np.zeros(shape=(len(df),), dtype=np.float32)  # array of size (n_event,)

# fill dataset
for event_n, event in df.iterrows():
    
    NodeOrder = event['NodeOrder']
    Nodededx = event['Nodededx'][NodeOrder]
    
    # retrieve the first node
    X[event_n] = Nodededx[0]
    # momentum label
    y[event_n] = event['TrueMomentum']

# standardize the dataset (mean=0, std=1)
X_stan = scale(X) 

In order to understand the training data, it's always good to visualise first. A good way of doing it could be creating a scatter plot of each of our variables against the momentum:

In [ ]:
param_names = ['momentum (MeV)', 'dE/dx node 1']
plot_parameters(X, y, param_names, y_names=None)

Unfortunately, it doesn't look like the input follows a linear distribution. Anyway, we could train a linear regression model on the input and see what it learns.

Training a machine-learning algorithm is usually not an easy task. The algorithm learns from some training data until it is ready to make predictions on unseen data. In order to test how the algorithm performs on new data, the datasets used for training are divided into two groups (sometimes are divided into three groups, but we're keeping two groups here for simplicity):

- Training set: the model learns from this set only. It must be the largest set.
- Test set: it is used to evaluate the model at the end of the training, only once it is fully trained. 

In this example, we keep 60% of the data for training and 40% for testing.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_stan, y, test_size=0.4, random_state=7) # 60% training and 40% test

##Linear regression

Now we can run the linear regression on the training set and print the coefficients learnt:

In [ ]:
reg = LinearRegression().fit(X_train, y_train)
print("B0: {}, B1: {}".format(reg.intercept_, reg.coef_[0]))

Great! Let's draw the regression line of the model according to the equation:

$$
\hat{y} = \beta_0 + \beta_1 \cdot x_1
$$

In [ ]:
param_names = ['momentum (MeV)', 'dE/dx node 1']
plot_regression(X_train, y_train, reg, param_names)

As shown before, the input variable of our dataset does not follow a linear distribution, meaning that applying a linear regression model to predict the momentum from our data is not ideal. To try solve this problem, polynomial regression arises, which is, to all intents and purposes, a linear regression, only that its normal form has been extended to accommodate higher-order polynomials. In other words, it consists of fitting a model that follows the form:

$$\hat{y} = \beta_0 + \beta_1 x + \beta_2 x^2 + \ldots + \beta_g x^g$$

that is, we transform the function to a polynomial of degree $g$.

To do this in `sklearn` we have the preprocessing tool `preprocessing.PolynomialFeatures` which allows us to transform any data set into a new data set with polynomial and interrelated features. Since our dataset has only one feature $x_1$, if we want a polynomial of degree 3, `PolynomialFeatures` will extend the features in our set resulting in $x_1$, $x_1^2$, $x_1^3$, plus the bias. Note the exponential growth of the data set for high-degree polynomials:

In [ ]:
X_poly = PolynomialFeatures(degree=3, include_bias=True).fit_transform(X_stan)

# we divide again into train and test
X_poly_train, X_poly_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.4, random_state=7) # 60% training and 40% test

Let's run the linear regression again and plot the resulted line:

In [ ]:
reg = LinearRegression().fit(X_poly_train, y_train)

param_names = ['momentum (MeV)', 'dE/dx node 1']
plot_regression(X_train, y_train, reg, param_names, degree=3)

The results are still not ideal. It kind of makes sense taking into account our dataset is underused (we are using only one input variable).

A more robust but straightforward way of making the input data interpretable for the algorithm is to keep the information of only a few nodes of each track. Our preprocessing is illustrated in the following figure (there are many combinations, we are showing just one practical example here):

<div>
<img src="https://raw.githubusercontent.com/saulam/neutrinoml/main/reg.png" width="500"/>
</div>

where we keep the dE/dx of the first 3 and last 5 nodes of each track, along with their 4 global parameters, building up an array of size 12. For events where the track has less than 8 nodes (first 3 + last 5 nodes), we simply fill the empty positions of the array with -1s.

To sum up, with this preprocessing, we should end up having our input dataset **X**, consisting of 59,578 vectors of size 12 each (a 59,578x12 matrix). The values to estimate, **y**, are the momentums of each event. Besides, it's always recommended to shuffle the training examples to prevents any bias during the training.

In [ ]:
X = np.zeros(shape=(len(df),12), dtype=np.float32) # array of size (n_event, 12)
y = np.zeros(shape=(len(df),), dtype=np.float32)   # array of size (n_event,)
X.fill(-1) # filled with -1s

# fill dataset
for event_n, event in df.iterrows():
    
    NodeOrder = event['NodeOrder']
    Nodededx = event['Nodededx'][NodeOrder]
    
    # retrieve up to the first 3 nodes
    nfirstnodes = min(Nodededx.shape[0], 3)
    X[event_n,:nfirstnodes] = Nodededx[:nfirstnodes]
    
    if Nodededx.shape[0]>nfirstnodes:
        # retrieve up to the last 5 nodes
        nlastnodes = min(Nodededx.shape[0]-3, 5)
        X[event_n,nfirstnodes:nfirstnodes+nlastnodes] = Nodededx[-nlastnodes:]

    # global parameters
    X[event_n,-4] = event['TrkLen']
    X[event_n,-3] = event['TrkEDepo']
    X[event_n,-2] = event['TrkDir1']
    X[event_n,-1] = event['TrkDir2']
    
    # momentum label
    y[event_n] = event['TrueMomentum']

# standardize the dataset (mean=0, std=1)
X_stan = scale(X) 

# we divide again into train and test
X_train, X_test, y_train, y_test = train_test_split(X_stan, y, test_size=0.4, random_state=7) # 60% training and 40% test

In order to understand the training data, it's always good to visualise first. A good way of doing it could be creating a scatter plot of each of our 12 variables against the momentum:

In [ ]:
param_names = ['momentum (MeV)', 'dE/dx node 1', 'dE/dx node 2', 'dE/dx node 3', 'dE/dx node n-4',\
               'dE/dx node n-3', 'dE/dx node n-2', 'dE/dx node n-1', 'dE/dx node n', 'TrkLen',\
               'TrkEDepo', 'TrkDir1', 'TrkDir2']
plot_parameters(X, y, param_names, y_names=None)

As shown above, none of the 12 variables seems to follow a linear distribution. However, let's give it a try to linear regression again:

In [ ]:
reg = LinearRegression().fit(X_train, y_train)

The model learnt 12 coefficients ($\beta_1, \beta_2, ..., \beta_{12}$), one for each input parameter, plus the bias ($\beta_0$):
$$
Y = \beta_0 + \beta_1 \cdot X_1 + \beta_2 \cdot X_2 + \cdots + \beta_{12} \cdot X_{12} + \epsilon.
$$
We can print the values of the above coefficients:

In [ ]:
print("B0 (bias): {:2.5}".format(float(reg.intercept_)))
for i, coef in enumerate(reg.coef_):
    print("B{} (\"{}\"): {:2.4}".format(i+1, param_names[i+1],coef))

Now, it's time to run our regression model on the test set. The function `plot_distributions` shows some results.

In [ ]:
y_pred = reg.predict(X_test)
plot_distributions(X_test, y_test, y_pred)

The regression model has clearly learnt something sensible. However, it looks like the distribution is slightly shifted to the left; thus, the difference between the original and predicted values is not symmetrical.

As shown before, the input variables of our dataset do not follow a linear distribution, meaning that applying a linear regression model to predict the momentum from our data is not ideal in theory. To solve this problem, polynomial regression arises, which is, to all intents and purposes, a linear regression, only that its normal form has been extended to accommodate higher-order polynomials. In other words, it consists of fitting a model that follows the form:

$$\hat{y} = \beta_0 + \beta_1 x + \beta_2 x^2 + \ldots + \beta_g x^g$$

that is, we transform the function to a polynomial of degree $g$.

To do this in `sklearn` we have the preprocessing tool `preprocessing.PolynomialFeatures` which allows us to transform any data set into a new data set with polynomial and interrelated features. For example, if our data set has only two features $x_1$ and $x_2$ and we want a polynomial of degree 3, `PolynomialFeatures` will extend the features in our set resulting in $x_1$, $x_2$, $x_1^2$, $x_2^2$, $x_1 \cdot x_2$, $x_1^3$, $x_2^3$, $x_1^2 \cdot x_2$ and $x_1 \cdot x_2^2$. Note the exponential growth of the data set for high-degree polynomials.

In [ ]:
X_poly = PolynomialFeatures(degree=3, include_bias=True).fit_transform(X_stan)

We split the dataset again into training and test sets:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.4, random_state=7) # 60% training and 40% test

And run the linear regression again on the transformed data:

In [ ]:
reg_poly = LinearRegression().fit(X_train, y_train)

Let's plot the results again:

In [ ]:
y_pred = reg_poly.predict(X_test)

plot_distributions(X_test, y_test, y_pred)

The results look more symmetrical now! The predicted distribution is not that wider, and we reduced the mean and the standard deviation of the difference between the original and the predicted values. 

Now it's your turn to try to beat these results! You may play with the dataset generation (using a different configuration of variables), preprocess the data differently, use another degree for the polynomial, etc.

##Homework

It's your time to beat the results above!

You could try to generate a new dataset based on your physics knowledge (i.e., influence the feature selection), or squeeze the regression algorithm.